# Bayes Examples

In this notebook we will demonstrate how to calculate probabilities and distributions directly from our data. We will do this *by hand* rather than using the scikit-learn functions, just because then we know what is happening.

# Imports

These are standard imports for working with the data and loading our Google Drive (which contains our CSV file) -- if your data is stored on your local PC, you'll have to edit this accordingly.

In [ ]:
import os
import pandas as pd
import numpy as np

# We use two different plotting libraries, depending on which kind of plot we want
import matplotlib.pyplot as plt

# Set an option for Pandas to display smaller floating-point numbers
pd.options.display.float_format = '{:,.2f}'.format

# Turn off warnings
import warnings
warnings.filterwarnings("ignore")

# Need to get Google Drive access
from google.colab import drive
drive.mount('/content/gdrive')

# Data Loading and Preprocessing

## Peeking at the Data

Remember, you should always quickly double-check that your data was loaded successfully by taking a peek at what your dataframe contains.

In [ ]:
# Load the dataset into a Pandas dataframe
data_dir = os.path.join('/content/gdrive/My Drive/classes/be432-2021/notebooks/wisconsin_breast_cancer_data.csv')
df = pd.read_csv(data_dir)

In [ ]:
df.head()

## Label Encoding

We create a new column that contains numeric values for each of the unique classes in our dataset.

"Malignant" is coded as "M" in the original data, and 1 in the new label column; "Benign" is "B" in the original, and 0 in the encoded data -- double-check to be sure! 

Note that the numeric values of the labels are assigned alphabetically -- there's no way for the program to know that "Malignant" is normally called "positive" (i.e. 1) and "Benign" is "negative" (0). It just so happens that B comes before M in the alphabet and it starts from 0.

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

diagnosis_cat = df['diagnosis']

# Fit the encoder to the categories, and immediately 
diagnosis_lab = label_encoder.fit_transform(diagnosis_cat)

# Add the diagnosis label back to the dataframe
df['diagnosis_label'] = diagnosis_lab

df.head(20)

## Data Splitting

Before we do anything else, we create our training and testing sets. We set a test size of 30%, and split on the diagnosis label value.

In [ ]:
# Stratified Split
from sklearn.model_selection import StratifiedShuffleSplit

# Create the splitting object
split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)

# Apply the split to the data frame using the "diagnosis" column as our label
for train_index, test_index in split.split(df, df["diagnosis_label"]):
    train_set = df.loc[train_index]
    test_set = df.loc[test_index]

In [ ]:
test_set.head(10)

For simplicitly, we separate out the feature values from the diagnosis label. We don't actually care about the patient IDs at this point (although if you're doing a more involved ML project, you probably want to trace back from your misclassifications to your sample IDs).

We also separate the values into separate class-specific dataframes as well; this helps to plot things out later on.

In [ ]:
training_values = train_set.drop(['id','diagnosis', 'diagnosis_label'], axis=1)
training_labels = train_set[['diagnosis_label']].copy()

testing_values = test_set.drop(['id','diagnosis', 'diagnosis_label'], axis=1)
testing_labels = test_set[['diagnosis_label']].copy()

In [ ]:
# Separate out our testing data into classes for easier plotting
malignant = training_values.loc[training_labels['diagnosis_label'] == 1,:]
benign = training_values.loc[training_labels['diagnosis_label'] == 0,:]

In [ ]:
benign.head()

# Distributions

In this section we create the various distributions that we'll be using to calculate probabilities.

First, select a feature to look at. We'll only look at a single feature in this example.

In [ ]:
feat_name = 'texture_mean'

# Create a nice-looking name to use in the plot
feat_display = feat_name.replace('_', ' ').title()

## Create a Density Histogram

Only four lines of code below are used to actually calculate the distribution: `ax.hist(...)` does most of the work, while the `np.append(...)` function is necessary to make the number of counts and bins match up.

Each `ax.hist(...)` line returns the following:

- `counts` is the number of samples in each bin of the histogram.
- `bins` contains the edges of each bin; the first value is the left-hand side of the left-most bin, and the last one is the right-hand side of the right-most bin. 

The rest of the code is justcreating and styling the resulting plot.

In [ ]:
# Create the holder for the plots
f, ax = plt.subplots(figsize=(10,6))

m_counts, m_bins, _ = ax.hist(malignant[feat_name], density=True, alpha=.8, label="Malignant", bins=10, edgecolor='k')
b_counts, b_bins, _ = ax.hist(benign[feat_name], density=True, alpha=.8, label="Benign",  bins=10, edgecolor='k')

# We need to pad counts with a zero because bins is larger by 1
m_counts = np.append(m_counts, [0])
b_counts = np.append(b_counts, [0])

#ax.set(xlim=(5,40))

# Annotate Plot
ax.set(xlabel=r'$x$: '+feat_display,
       ylabel=r'$p(x|\omega_{j})$',
       title='Probability Density Function (PDF) for '+feat_display)

ax.legend(frameon=True)
ax.grid(linestyle=':')
plt.tight_layout()

plt.show()

Note that on the `y`-axis, we are plotting $p(x|\omega_{i})$ rather than the actual number of samples that fall into that bin; this is because we passed `density=True` to the `ax.hist(...)` function.

## Examine Histogram Counts

Let's take a look at what the histogram actually calculates. This is the *percentage* of samples (because we asked for a density) in each class that fall within each bin.

In [ ]:
# Examine the outputs
print(f'Number of samples in each bin:')
print(f'\t\tBin\tCount')
print(f'')
for m_count, m_bin in zip(m_counts, m_bins):
  print(f'Malignant\t{m_bin:2.2f}\t{m_count:1.3f}')

print(f'='*29)

print(f'\t\tBin\tCount')
print(f'')
for b_count, b_bin in zip(b_counts, b_bins):
  print(f'Benign\t\t{b_bin:2.2f}\t{b_count:1.3f}')

# Calculating Probabilities

## Probability Density Function

Let's try to calcualte $p(x|\omega_{i})$ for both $\omega_{0}$ (Benign) and  $\omega_{1}$ (Malignant) for a *specific* value of $x$.

For example, what if we measure a feature value of 20? What's the probability of observing a 20 from the benign vs. the malignant classes?

In [ ]:
feature_value = 20
#feature_value = 0.07

# First we find the bin associated with this value
# This will be the first bin that is greater than our target value
m_locs = m_bins > feature_value
m_idx = np.min(np.where(m_locs))

print(f'Feature value {feature_value} appears in the {m_idx}th bin of the Malignant histogram.')

b_locs = b_bins > feature_value
b_idx = np.min(np.where(b_locs))

print(f'Feature value {feature_value} appears in the {b_idx}th bin of the Benign histogram.')

Now, we can just look at the corresponding bin in each histogram and pull the percentage of samples in each class that have that feature value.

In [ ]:
p_x_given_m = m_counts[m_idx]
p_x_given_b = b_counts[b_idx]

In [ ]:
from IPython.display import Math, HTML
display(HTML("<script src='https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.3/"
               "latest.js?config=default'></script>"))
Math(r'$p(x|\omega_{1}) = '+f'{p_x_given_m:2.3f}'+r'$')

In [ ]:
from IPython.display import Math, HTML
display(HTML("<script src='https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.3/"
               "latest.js?config=default'></script>"))
Math(r'$p(x|\omega_{2}) = '+f'{p_x_given_b:2.3f}'+r'$')

### Refactor to a Function

Let's create a function that will return the histogram counts (density) for any value. 

This is *very common* in programming -- whenever you anticipate you'll have to do something more than once, it's a good idea to write a function to do it for you. It takes more time up front, but saves a ton of time later on.

In [ ]:
def p_x_given_omega(bins, counts, value):
  """Given a set of bins and corresponding histogram counts, finds the counts associated with a specific value."""
  
  assert len(bins) == len(counts), f'Bins has length of {len(bins)}, counts has length of {len(counts)}; these should match.'
  locs = bins > value

  # If none of the locations are larger than the value, then return zero
  if np.any(locs):
    idx = np.min(np.where(locs))
    return counts[idx]
  else:
    return 0

## Prior Probabilities

The prior in this case is "what is the likelihood that we observe a benign or malignant case, regardless of the feature data?"

This is often difficult to calculate. For our case, we'll just set them up as non-informative priors:

$$p(\omega_{0}) = p(\omega_{1}) = 0.5$$

In [ ]:
# Prior probabilities
p_m = 0.5
p_b = 1 - p_m

## "Evidence" or "Normalizing Factor"

The *a posteriori* probability must be normalized. To do this, we sum up all the possible ways we could observe a particular feature value:

1. When the class is $\omega_{0}$; or 
2. When the class is $\omega_{1}$

To do this, we first need to set up the x-axis that will be our feature "domain", or the set of values that we want to calculate over. Mathematically this should be $-\infty$ to $+\infty$, but for our case we can just go from the minimum observed feature value to the maximum feature value.

In [ ]:
# Set up the bounds of our domain by calculating the min and max feature values
x_min = np.min([malignant[feat_name].min(), benign[feat_name].min()])
x_max = np.max([malignant[feat_name].max(), benign[feat_name].max()])

print(f'Minimum value for {feat_name}: {x_min:2.2f}')
print(f'Maximum value for {feat_name}: {x_max:2.2f}')

In [ ]:
# Now create a vector that goes from min to max, representing different feature values we could measure.
x_domain = np.arange(x_min, x_max, 0.001)

# Our step size is 0.01, which should be good enough for our purposes.
print(f'X domain for {feat_name}:')
print(x_domain)
print('')
print(f'Number of samples in our x axis: {len(x_domain)}')

Now for each of these $x$ values, calculate our denominator value:

$$ \sum_{i=0}^{c} P(x|\omega_{i})P(\omega_{i}) $$

In [ ]:
denominator = []
for x in x_domain:
  denominator.append(p_x_given_omega(m_bins, m_counts, x)*p_m + p_x_given_omega(b_bins, b_counts, x)*p_b)

## *A Posteriori* Probability of Observing $\omega_{i}$

Finally we can calculate $P(\omega_{i} | x)$ for each value of $x$ in our domain.

In [ ]:
p_m_given_x = []
p_b_given_x = []
for i,x in enumerate(x_domain):
  p_m_given_x.append( (p_x_given_omega(m_bins, m_counts, x) * p_m) / denominator[i] )
  p_b_given_x.append( (p_x_given_omega(b_bins, b_counts, x) * p_b) / denominator[i] )

In [ ]:
# Create the holder for the plots
f, ax = plt.subplots(figsize=(10,6))


ax.plot(x_domain, p_m_given_x, alpha=0.6, linewidth=5, label=r'$p(\omega_{1} | x)$ (Malignant)')
ax.plot(x_domain, p_b_given_x, alpha=0.6, linewidth=5, label=r'$p(\omega_{0} | x)$ (Benign)')

#ax.set(xlim=(5,40))
ax.set(xlim=(x_domain.min(), x_domain.max()))

# Annotate Plot
ax.set(xlabel=r'$x$ domain for feature: '+feat_display,
       ylabel=r'$p(\omega_{j} | x)$',
       title='A Posteriori Probability for '+feat_display)

ax.legend(frameon=True)
ax.grid(linestyle=':')
plt.tight_layout()

plt.show()

Note that this output is very "blocky" -- that's because for all of the samples within a bin, we have the same output value.

What were to happen if we use a **parametric** probability density function?

# The Cheaty Way

Of course, you could just... cheat.

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(np.array(training_values[feat_name]).reshape(-1, 1), training_labels).predict_proba(x_domain.reshape(-1, 1))

In [ ]:
y_pred

In [ ]:
# Create the holder for the plots
f, axes = plt.subplots(1,2,figsize=(20,6))

axes[0].plot(x_domain, p_m_given_x, alpha=0.6, linewidth=5, label=r'$p(\omega_{1} | x)$ (Malignant)')
axes[0].plot(x_domain, p_b_given_x, alpha=0.6, linewidth=5, label=r'$p(\omega_{0} | x)$ (Benign)')

axes[1].plot(x_domain, y_pred[:,1], alpha=0.6, linewidth=5, label=r'$p(\omega_{1} | x)$ (Malignant)')
axes[1].plot(x_domain, y_pred[:,0], alpha=0.6, linewidth=5, label=r'$p(\omega_{0} | x)$ (Benign)')


for ax in axes:
  ax.set(xlim=(5,40))

  # Annotate Plot
  ax.set(xlabel=r'$x$ domain for feature: '+feat_display,
        ylabel=r'$p(\omega_{j} | x)$',
        title='A Posteriori Probability for '+feat_display)

  ax.legend(frameon=True)
  ax.grid(linestyle=':')
  plt.tight_layout()

plt.show()

What are the similarities between these curves? What are the differences?